# Setting

In [1]:
import sys, os
import pandas as pd

sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")
sys.path.append("../../../../")
sys.path.append("../../../../../")

from Clust.setting import influx_setting_KETI as ins
from Clust.clust.ingestion.influx import influx_client_v2 as iC
from Clust.clust.ingestion.mongo import mongo_client
from Clust.clust.ML.common import ML_api

db_client = iC.InfluxClient(ins.CLUSTDataServer2)
mongodb_client = mongo_client.MongoClient(ins.CLUSTMetaInfo2)

cpu is available.


In [2]:
# feature list와 data_y_flag는 model_info에서 받아와야 합니다.
# regression
param1 = {
    "ingestion_param_X" :{
        "bucket_name": 'integration',
        "ms_name" : 'regression_energy_cleanLevel4_testX',
        "feature_list":['Press_mm_hg', 'RH_1', 'RH_2', 'RH_3', 'RH_4', 'RH_5', 'RH_6', 'RH_7',
       'RH_8', 'RH_9', 'RH_out', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7',
       'T8', 'T9', 'T_out', 'Tdewpoint', 'Visibility', 'Windspeed']
    },
    'model_name':'regression_energy_cleanLevel4_trainX_regression_LSTM_rg_'
}

# forecasting
param2 = {
    "ingestion_param_X" :{
        "bucket_name": 'integration',
        "ms_name" : 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_testX',
        "feature_list":['Temperature', 'out_temp','sin_hour']
    },
    'model_name':'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_trainX_regression_LSTM_rg_'
}

# classification
param3 = {
    "ingestion_param_X" :{
        "bucket_name": 'integration',
        "ms_name" : 'classification_actionPattern_cleanLevel0_testX',
        "feature_list":['col_0', 'col_1','col_2','col_3','col_4','col_5','col_6','col_7','col_8']
    },
    'model_name':'classification_actionPattern_cleanLevel0_trainX_classification_LSTM_cf_'
}

In [3]:
params = param3

# chage tpye string to bool -> ex) 'true' -> True
params = ML_api.chagne_type_str_to_bool(params)

In [4]:
# get model meta
model_name = params['model_name']


meta_file_name = "./meta.json"
from Clust.clust.ML.tool import meta
#model_meta = meta.read_model_meta_from_mongodb(mongodb_client, 'model', 'meta', model_name)
model_meta = meta.read_model_meta_from_local(meta_file_name, model_name)

In [5]:
model_meta

{'ingestion_param_X': {'bucket_name': 'integration',
  'ms_name': 'classification_actionPattern_cleanLevel0_trainX',
  'feature_list': ['col_0',
   'col_1',
   'col_2',
   'col_3',
   'col_4',
   'col_5',
   'col_6',
   'col_7',
   'col_8']},
 'ingestion_param_y': {'bucket_name': 'integration',
  'ms_name': 'classification_actionPattern_cleanLevel0_trainy',
  'feature_list': ['value']},
 'data_y_flag': True,
 'scaler_param': {'scaler_flag': 'scale',
  'scale_method': 'minmax',
  'scaler_path': './scaler/',
  'scaler_file_path': {'XScalerFile': {'fileName': 'scaler.pkl',
    'filePath': './scaler/classification_actionPattern_cleanLevel0_trainX/minmax/2b219d77d5b069971addb3d7c728ec0d/scaler.pkl'},
   'yScalerFile': {'fileName': 'scaler.pkl',
    'filePath': './scaler/classification_actionPattern_cleanLevel0_trainy/minmax/f69156750a210491ffd4a67b605bc88b/scaler.pkl'}}},
 'transform_param': {'data_clean_option': False,
  'split_mode': 'window_split',
  'nan_process_info': {'type': 'num',
 

In [6]:
# make test data, scaler
infer_X, scaler_X, scaler_y = ML_api.infer_data_preparation(params, model_meta, db_client)

/home/leezy/anaconda3/envs/clust/lib/python3.8/site-packages/sklearn/base.py:299: UserWarning: Trying to unpickle estimator MinMaxScaler from version 1.2.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/leezy/anaconda3/envs/clust/lib/python3.8/site-packages/sklearn/base.py:299: UserWarning: Trying to unpickle estimator MinMaxScaler from version 1.2.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [7]:
infer_X.shape

(1385, 9, 128)

In [8]:
# model inference
prediction_result = ML_api.ML_inference(model_meta, infer_X, scaler_X, scaler_y)


Start inference

** Dimension of result for inference dataset = (1392,)
1392
1


In [19]:
prediction_result

,value
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
1387,0.0
1388,0.0
1389,0.0
1390,0.0


In [ ]:
prediction_result[prediction_result['value']==x]